In [5]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import squareform
from scipy.spatial.distance import cosine
import psycopg2
from scipy import sparse
import sklearn.metrics.pairwise as pw
from sklearn.cross_validation import train_test_split

connection = psycopg2.connect(user="postgres",
                              password="aleixDatabase",
                              host="localhost",
                              port="3306",
                              database="siodb")

cursor = connection.cursor()

median_query = '''SELECT username, restaurant, rating FROM ratings'''

cursor.execute(median_query)
ratings = cursor.fetchall()

all_ratings = pd.DataFrame(ratings, columns=['userID', 'restaurantID', 'rating'])
all_ratings['index'] = all_ratings['userID'].copy()
all_ratings.set_index('index', inplace = True)

user_ratings = all_ratings.drop(['restaurantID'], axis=1)
user_ratings = user_ratings.groupby('userID', as_index=False).mean()

n_users = all_ratings.userID.unique().shape[0]
n_users

n_items = all_ratings.restaurantID.unique().shape[0]
n_items

filter_users = all_ratings['userID'].value_counts() > 90
filter_users = filter_users[filter_users].index.tolist()

filter_restaurants = all_ratings['restaurantID'].value_counts() > 68000
filter_restaurants = filter_restaurants[filter_restaurants].index.tolist()

user_ratings_pos = user_ratings[user_ratings['rating'] > 0].reset_index(drop = True)
all_ratings_pos = all_ratings[all_ratings['userID'].isin(user_ratings_pos['userID'])]

pivot_ratings = all_ratings_pos.pivot_table(index=['userID'], columns=['restaurantID'], values='rating')

reduced_pivot = pivot_ratings.copy()

nan_pivot = reduced_pivot.fillna(0)



ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [4]:
trainset, testset = train_test_split(nan_pivot, test_size=0.25)
matrix = sparse.csr_matrix(trainset)
matrix


NameError: name 'train_test_split' is not defined

In [ ]:
sim_matrix = 1 - pw.cosine_distances(matrix)